<a href="https://colab.research.google.com/github/unisttt/open-implementation/blob/master/Titanic_submit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
* 二つの手法を実装し検証
* （手法１）sckit-learnのLogisticRegression
 * cross validationが便利そうなので使用してみる
 * 平均正解率: {0.8011819166748744}
 * 短いコードで実装できる
* （手法2）PyTorch + skorchで実装
 * PyTrochでsckit-learnのcross validationが使えるみたいなので試してみる
 * 平均正解率: {0.8207426376440461}
 * コードは長くなるが複雑な問題に適応できる

## 事前準備

### GPUを使う場合の設定（PyTorchで"CUDA"を使いたい場合）
1. 上のタブ"ランタイム"タブ選択
2. "ランタイムのタイプを変更を選択"
3. "ハードウェアアクセラレータ"を"GPU"に設定

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

### ファイルアップロード
1.   左の”＞”マークを開く
2.   ”ファイル”を選択
3.   ”アップロード”を選択
4.   "train.csv"を選択

In [0]:
# アップロードされていない場合にはここでアップロードしてもらう
if not os.path.exists("train.csv"):
  from google.colab import files
  uploaded = files.upload()    # train.csvを選択

In [0]:
df_train = pd.read_csv("train.csv")

In [4]:
# 欠落と割合を表示
def nullCheck(df):
    null_count = df.isnull().sum()
    null_rate = null_count / df.shape[0] * 100

    null_table = pd.DataFrame({
        'null_couunt': null_count,
        'null_rate': null_rate
    })
    
    return null_table

print(nullCheck(df_train))

             null_couunt  null_rate
PassengerId            0   0.000000
Survived               0   0.000000
Pclass                 0   0.000000
Name                   0   0.000000
Sex                    0   0.000000
Age                  177  19.865320
SibSp                  0   0.000000
Parch                  0   0.000000
Ticket                 0   0.000000
Fare                   0   0.000000
Cabin                687  77.104377
Embarked               2   0.224467


## 前処理

* PassengerId: / （いらなそう）
* Survived: / （train_y）
* Pclass : Ticket Class(1st, 2nd, 3rd) / （使えそう）
* Name: / （いらない）
* Sex: / （絶対いる）
* Age: /（ 絶対いる）
* SibSp : タイタニック号に乗っている兄弟/配偶者の数 / （一応関係ありそう）
* Parch : タイタニック号に乗っている親/子供の数 / （関係ありそう）
* Ticket : Ticket number / （いらない）
* Fare: / （いるはず）
* Cabin : Cabin number / （いらないはず。欠落が多すぎる）
* Embarked : 乗船港（C = Cherbourg, Q = Queenstown, S = Southampton）/ （あんまり関係なさそう）

In [5]:
# 使用しないデータを削除する（直観で選択）
# Ageの欠落を平均などで埋めれるが、正確な年齢が重要な気がするので今回はしない。欠落は削除
df_train_use = df_train.drop(["PassengerId", "Name", "Ticket", "Cabin", "Embarked"], axis=1).dropna()
print(df_train_use.shape)

(714, 7)


In [6]:
df_train_use.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


#### "male", "female"の二つの内一つは必要ないので削除（Pclassでも同様）
正解率が少し上がった

In [0]:
def createMergeDataset():
  Pclass = pd.get_dummies(df_train_use["Pclass"])
  Pclass.columns = ["1st", "2nd", "3rd"]
  Pclass = Pclass.drop("1st", axis=1)

  Sex = pd.get_dummies(df_train_use["Sex"])
  Sex = Sex.drop("female", axis=1)

  tmp_data = df_train_use.drop(["Pclass", "Sex"], axis=1)
  Merge_data = pd.merge(tmp_data, Pclass, right_index=True, left_index=True)
  Merge_data = pd.merge(Merge_data, Sex, right_index=True, left_index=True)
  Merge_data.head()
  
  return Merge_data

In [8]:
def createDataset(Merge_data):
  train_x = Merge_data.drop("Survived", axis=1).values
  train_y = Merge_data["Survived"].values
  print(train_x.shape)
  print(train_y.shape)
  return train_x, train_y

Merge_data = createMergeDataset()
train_x, train_y = createDataset(Merge_data)
m, n = train_x.shape

(714, 7)
(714,)


## （手法1）sckit-learnのLogisticRegressionで実装

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import random

In [10]:
random.seed(0)  # 恐らくこれを入れることでランタイムをリセットしても値が変わらなくなるはず
model = LogisticRegression(solver="lbfgs", max_iter=500)

# 層化 k 分割交差検証
stratifiedkfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
score = cross_val_score(model, train_x, train_y, cv=stratifiedkfold)
print('Cross-validation scores: \n{}'.format(score))
print("Average: {}".format(np.mean(score)))

# random_state=0 に変更
stratifiedkfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
score = cross_val_score(model, train_x, train_y, cv=stratifiedkfold)
print('Cross-validation scores: \n{}'.format(score))
print("Average: {}".format(np.mean(score)))

Cross-validation scores: 
[0.73426573 0.85314685 0.82517483 0.75524476 0.78873239]
Average: 0.7913129124396729
Cross-validation scores: 
[0.76923077 0.77622378 0.7972028  0.81818182 0.84507042]
Average: 0.8011819166748744


In [11]:
def valifyAverageScore(score):
  if score >= 0.8:
    print("score: {}\n平均正解率は0.8以上です.".format(score))
  elif 0.75 <= score < 0.8:
    print("score: {}\n平均正解率は0.75以上0.8未満です.".format(score))
  else:
    print("score: {}\n平均正解率は0.75未満です.".format(score))

valifyAverageScore(np.mean(score))

score: 0.8011819166748744
平均正解率は0.8以上です.


## （手法2）PyTorch + skorchで実装

skorch: PyTorchのsklearnラッパー<br>
sklearnの関数が使える

In [12]:
!pip install skorch

     |████████████████████████████████| 102kB 4.4MB/s 


In [0]:
import skorch

In [0]:
import torch
from torch import nn
import torch.nn.functional as F

# シンプルな実装
# 隠れ層は入力の3倍
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(7, 28)
    self.fc2 = nn.Linear(28, 2)
    
  def forward(self, x):
    x = self.fc1(x)
    x = F.dropout(x, p=0.1)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.softmax(x, dim=-1)
    
    return x

In [0]:
from skorch import NeuralNet
from skorch import NeuralNetClassifier

device = "cuda" if torch.cuda.is_available() else "cpu"

model = NeuralNetClassifier(
  Net,
  max_epochs=10,
  optimizer=torch.optim.Adam,
  lr=0.01,
  device=device,
  batch_size=50,
  criterion=nn.NLLLoss,
  train_split=None
)

In [0]:
torch.manual_seed(1)  # 恐らく実行毎にパラメータの初期値が変わらなくなる

train_x = train_x.astype(np.float32)
train_y = train_y.astype(np.int64)

#### 標準化することでtrain_lossの値が時々nanになるのを解消できた
その分正解率も上がった

In [17]:
# 標準化
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train_x_std = sc.fit_transform(train_x)
print(train_x[1])
print(train_x_std[1])

[38.      1.      0.     71.2833  0.      0.      0.    ]
[ 0.571831    0.5245701  -0.50589514  0.6918968  -0.56548935 -0.9944134
 -1.317434  ]


In [18]:
random.seed(1)
torch.manual_seed(1)

average_hist = []  # 検証用
model.max_epochs = 50

# 層化 k 分割交差検証
stratifiedkfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
score2 = cross_val_score(model, train_x_std, train_y, cv=stratifiedkfold)
print('Cross-validation scores: \n{}'.format(score2))
print("Average: {}".format(np.mean(score2)))
average_hist = np.append(average_hist, np.mean(score2))

  epoch    train_loss     dur
-------  ------------  ------
      1        0.6108  0.3667
      2        0.4825  0.0273
      3        0.4323  0.0268
      4        0.4224  0.0234
      5        0.4082  0.0223
      6        0.4093  0.0236
      7        0.4060  0.0223
      8        0.4065  0.0225
      9        0.3999  0.0282
     10        0.3974  0.0214
     11        0.4037  0.0264
     12        0.3995  0.0276
     13        0.3934  0.0218
     14        0.3912  0.0251
     15        0.3956  0.0217
     16        0.3901  0.0240
     17        0.3946  0.0277
     18        0.3966  0.0279
     19        0.3943  0.0222
     20        0.3923  0.0235
     21        0.3857  0.0251
     22        0.3800  0.0204
     23        0.3892  0.0259
     24        0.3937  0.0219
     25        0.3862  0.0233
     26        0.3796  0.0235
     27        0.3900  0.0210
     28        0.3829  0.0230
     29        0.3966  0.0263
     30        0.3823  0.0229
     31        0.3865  0.0238
     32   

In [19]:
valifyAverageScore(np.mean(score2))

score: 0.8207426376440461
平均正解率は0.8以上です.


#より精度を上げるために考えられること
* 前処理をもっとやる
 * 年齢を分ける
 * 新しい特徴を加える（親, 独身, etc）
 
* （手法2）他の構成を検証する
 * 隠れ層の数、ノードの数
 * Activation Functionを変更する
 * epoch数などの検証
 
* 他の手法を試す
 * 決定木?
 * ベイジアンネットワーク
 * SVM

## 感想
この課題を通して、これまで自分がインプットばかりでアウトプットが全くできていなかったということに気づかされました。理論的には理解しているつもりでも実際に取り掛かってみると、思わぬところで躓いてしまったりして時間が掛かってしまいました。とくに前処理についてはこれまで蔑ろにしてきたところなので、きちんと学ぶ必要性を感じました。インターンの開始まではまだ時間があるため、他のデータセットや他の手法などにも挑戦してみたいと思います。